<a href="https://colab.research.google.com/github/vatsaaa/mtech/blob/main/semester_1/03_assignments/mfml/Assignment_01_Q01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python function to create a random matrix, but without using libraries

In [4]:
import random

In [5]:
def generate_random_matrix(rows: int, columns: int, decimal_allowed: bool=False):
    matrix = []

    for _ in range(rows):
      if decimal_allowed:
        # Chose 100 and 990 so that the matrix printing is better to look at
        row = [random.choice([random.randint(100, 999), round(random.uniform(0, 1), 2)]) for _ in range(columns)]
      else:
        # Chose 100 and 990 so that the matrix printing is better to look at
        row = [random.randint(100, 999) for _ in range(columns)]

        matrix.append(row)

    return matrix

In [8]:
two_by_two = generate_random_matrix(2, 2, False)

two_by_two

[[334, 421], [116, 316]]

The above generated matrix is in a single line, that really does not look like a matrix. So, I create my own function to print a matrix that looks like a matrix.

In [72]:
def print_matrix(matrix):
    print("[", end="")
    for i, row in enumerate(matrix):
        print(row, end="")
        if i != len(matrix) - 1:
            print(",\n", end="")
    print("]")

In [73]:
print_matrix(two_by_two)

[[334, 421],
[116, 316]]


Now we get a two_by_one matrix, before creating the augmented matrix

In [70]:
two_by_one = generate_random_matrix(2, 1, False)

print_matrix(two_by_one)

[228],
[335]

Below function will create the augmented matrix required by Gaussian Elimination when

In [76]:
def construct_augmented_matrix(A, b):
    if len(A) != len(b):
        raise ValueError("Number of rows in A must be equal to the length of b")

    # Zip A and b to create the augmented matrix
    augmented_matrix = [row + bi for row, bi in zip(A, b)]

    return augmented_matrix

In [77]:
A = two_by_two
b = two_by_one

augmented_matrix_A_b = construct_augmented_matrix(A, b)

print_matrix(augmented_matrix_A_b)

[[334, 421, 228],
[116, 316, 335]]


Function to find row echelon form of a given matrix

In [79]:
from math import floor

def find_row_echelon_form(matrix):
    rows = len(matrix)
    columns = len(matrix[0])

    pivot = 0
    for row in range(rows):
        if pivot >= columns:
            break

        # When pivot is 0 find a row where pivot element is not 0 and
        if matrix[row][pivot] == 0:
          for r in range(row + 1, rows):
            if matrix[r][pivot] != 0:
              # swap current row and row with non-zero pivot element
              matrix[row], matrix[r] = matrix[r], matrix[row]
              break
            else:
              pivot += 1

        # Now, when pivot position is not 0, do elementary row transformation
        # so as to ensure below the pivot we get zeros in the entire column
        for r in range(row + 1, rows):
          factor_nr = float(matrix[r][pivot])
          factor_dr = float(matrix[row][pivot])
          factor = factor_nr / factor_dr
          for c in range(pivot, columns):
            a = float(matrix[r][c])
            b = float(matrix[row][c])
            matrix[r][c] = float(matrix[r][c]) - factor * float(matrix[row][c])

        pivot += 1

    return matrix


In [81]:
ref_A_b = find_row_echelon_form(augmented_matrix_A_b)

print_matrix(ref_A_b)

[[334, 421, 228],
[0.0, 169.78443113772454, 255.81437125748502]]


Function to check if a given matrix is in row echelon form

In [82]:
def is_row_echelon(matrix):
    rows = len(matrix)
    columns = len(matrix[0])

    pivot = 0
    for row in range(rows):
      if pivot >= columns:
        break

      # Check if the pivot element is zero
      if matrix[row][pivot] == 0:
        return False

      # Check if all elements below the pivot are zero
      for r in range(row + 1, rows):
        if matrix[r][pivot] != 0:
          return False

      pivot += 1

    return True

In [83]:
print("Given matrix is in REF") if is_row_echelon(ref_A_b) else print("Given matrix is not in REF")

Given matrix is in REF


Function to get reduced row echelon form of a matrix in row echelon form

In [88]:
def find_reduced_row_echelon_form(matrix, find_ref=False):
    if not is_row_echelon(matrix) and find_ref:
        matrix = find_row_echelon_form(matrix)
    elif not is_row_echelon(matrix) and not find_ref:
        raise Exception("Given matrix is not in row echelon form!")

    rows = len(matrix)
    columns = len(matrix[0])

    pivot = 0
    for row in range(rows):
        if pivot >= columns:
            break

        # Divide the pivot row by the pivot element
        pivot_element = matrix[row][pivot]
        for c in range(columns):
          matrix[row][c] /= pivot_element

        # Eliminate non-zero elements above the pivot
        for r in range(row):
            factor = matrix[r][pivot]
            for c in range(columns):
                matrix[r][c] -= factor * matrix[row][c]

        # Eliminate non-zero elements below the pivot
        for r in range(row + 1, rows):
            factor = matrix[r][pivot]
            for c in range(columns):
                matrix[r][c] -= factor * matrix[row][c]

        pivot += 1

    return matrix


In [89]:
rref_A_b = find_reduced_row_echelon_form(ref_A_b)

print_matrix(rref_A_b)

[[1.0, 0.0, -1.2165302955618245],
[0.0, 1.0, 1.5067009945686676]]
